In [4]:
'''
Pipeline issue to be resolved...
Exact same code here but it works
Run every week if needed

'''
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import urllib
from utils.marketinsights import process_tables, get_city_mapping, get_contact_information, get_embeddings, get_industry, get_intersection, get_phone_mapping, label_country_by_city, label_country_by_phone, final_country, convert_to_datetime, AutoModel, AutoTokenizer

model_name = 'nomic-ai/nomic-embed-text-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

df = pd.read_csv(latest_file)

df_temp = df['tables'].apply(lambda x: pd.Series(process_tables(x)))
df_temp.columns = ['Executives', 'Shareholders']
df[['Executives', 'Shareholders']] = df_temp

print('Labelling Country and Industry')

try:
    existing = pd.read_csv('utils/data/Scrape/phoneextensions.csv')
except:
    existing = pd.DataFrame()
phone_storage = get_phone_mapping(existing)
city_storage = get_city_mapping()
df['Industry'] = df['raw'].apply(get_industry)
df['Contact Information'] = df['raw'].apply(get_contact_information)
df['Country_phone'] = df['Contact Information'].apply(lambda x: label_country_by_phone(x, phone_storage))
df['Country_city'] = df['Contact Information'].apply(lambda x: label_country_by_city(x, city_storage))
df['Country_candidates'] = df.apply(lambda x: get_intersection(x.Country_phone, x.Country_city), axis = 1)
df['Country'] = df.apply(lambda x: final_country(x['Contact Information'], x.Country_candidates, tokenizer, model), axis = 1)

# Apply the conversion functions
df['Time'] = df['date'].apply(convert_to_datetime)
df.drop(['date'], axis = 1)
df.to_csv('utils/data/Scraped News/marketinsights_data_2024-07-11.csv')

/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>


['./utils/data/Scraped News/marketinsights_data_2024-07-15.csv']
./utils/data/Scraped News/marketinsights_data_2024-07-15.csv
Labelling Country and Industry


In [62]:
'''
Trying out sentiment scoring for article content
'''

from transformers import BertForSequenceClassification, BertTokenizer
import torch

test_content = df.iloc[5]['Article content']
print(test_content)

# initialize our model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')


def get_sentiment(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=False)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    # define our starting position (0) and window size (number of tokens in each chunk)
    start = 0
    window_size = 512
    
    # initialize probabilities list
    probs_list = []
    
    start = 0
    window_size = 510  # we take 2 off here so that we can fit in our [CLS] and [SEP] tokens
    
    loop = True
    
    while loop:
        end = start + window_size
        if end >= total_len:
            loop = False
            end = total_len
        # (1) extract window from input_ids and attention_mask
        input_ids_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]
        # (2) add [CLS] and [SEP]
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]
        # (3) add padding upto window_size + 2 (512) tokens
        input_ids_chunk += [0] * (window_size - len(input_ids_chunk) + 2)
        attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) + 2)
        # (4) format into PyTorch tensors dictionary
        input_dict = {
            'input_ids': torch.Tensor([input_ids_chunk]).long(),
            'attention_mask': torch.Tensor([attention_mask_chunk]).int()
        }
        # (5) make logits prediction
        outputs = model(**input_dict)
        # (6) calculate softmax and append to list
        probs = torch.nn.functional.softmax(outputs[0], dim=-1)
        probs_list.append(probs)
    
        start = end

    stacks = torch.stack(probs_list)
    shape = stacks.shape
    with torch.no_grad():
        # we must include our stacks operation in here too
        stacks = torch.stack(probs_list)
        # now resize
        stacks = stacks.resize_(stacks.shape[0], stacks.shape[2])
        # finally, we can calculate the mean value for each sentiment class
        mean = stacks.mean(dim=0)
    print(mean)
    winner = torch.argmax(mean).item()
    result = ['Positive', 'Negative', 'Neutral'][winner]
    return result

df['Sentiment'] = df['Article content'].apply(get_sentiment)
for i, row in df.iterrows():
    print('Content: \n')
    print(row['Article content'])
    print('\nSentiment: \n')
    print(row['Sentiment'])
    print('**********')

(Alliance News) - Edil San Felice Spa reported obtaining a tax credit of EUR467,000.
The benefit was obtained as a result of the company's application to incur consulting costs related to Edil San Felice's admission to a regulated market, Euronext Growth Milan.
Edil San Felice closed Friday's session in the red by 2.3 percent at EUR4.25 per share.
By Maurizio Carta, Alliance News reporter
Comments and questions to redazione@alliancenews.com
Copyright 2024 Alliance News IS Italian Service Ltd. All rights reserved.


/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Token indices sequence length is longer than the specified maximum sequence length for this model (1629 > 512). Running this sequence through the model will result in indexing errors


NameError: name 'total_len' is not defined

In [60]:
'''
Trying out labelling missing executives from company website
'''

from serpapi import GoogleSearch
import pandas as pd
import streamlit as st
import requests
import re

from serpapi import GoogleSearch
import urllib
from bs4 import BeautifulSoup

def search_company_management(company_name, api_key):
    # Define common terms used for management page
    management_terms = {
        "Our Team", "Leadership", "Directors", 
        "Management", "Board", "Organization"
    }
    
    # Construct query with various combinations
    queries = f"({company_name} company website) AND ({' OR '.join(management_terms)})"
    
    params = {
        "q": queries,
        "engine": "google",
        "google_domain": "google.com",
        "num": 10,
        "nfpr": 1,
        "api_key": api_key,
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Scraping each result page
    storage = []
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        }
    for temp in results['organic_results']:
        link = temp['link']
        try:
            response = requests.get(link, headers = header, timeout=10)
            response.raise_for_status()  # Raise an exception if the request was unsuccessful

            # Parse the HTML content of the page with BeautifulSoup
            html_content = urllib.parse.unquote(response.text)
            soup = BeautifulSoup(html_content, 'html.parser')
            # Extract and clean text content
            text_content = soup.get_text(separator=' ', strip=True)
            
            if text_content.strip():
                storage.append(text_content)
        except:
            print(f"Unable to scrape {link}")
    return storage, results

In [61]:
# Example usage
api_key = st.secrets['serp_api_key']
company_name = 'StubHub'
search_results, x = search_company_management(company_name, api_key)

Unable to scrape https://www.comparably.com/companies/stubhub/executive-team
Unable to scrape https://pitchbook.com/profiles/company/13230-37
Unable to scrape https://www.dnb.com/business-directory/company-profiles.stubhub_inc.30a2fb4082f9f3d30b41552d98df63ec.html


In [45]:
print(x)

{'search_metadata': {'id': '6695d9cfd5a5314680bbfecf', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/5c2e89eeb1961f3e/6695d9cfd5a5314680bbfecf.json', 'created_at': '2024-07-16 02:24:15 UTC', 'processed_at': '2024-07-16 02:24:15 UTC', 'google_url': 'https://www.google.com/search?q=%28StubHub+company+website%29+AND+%28Organization+OR+Board+OR+Our+Team+OR+Directors+OR+Leadership+OR+Management%29&oq=%28StubHub+company+website%29+AND+%28Organization+OR+Board+OR+Our+Team+OR+Directors+OR+Leadership+OR+Management%29&num=10&nfpr=1&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/5c2e89eeb1961f3e/6695d9cfd5a5314680bbfecf.html', 'total_time_taken': 4.53}, 'search_parameters': {'engine': 'google', 'q': '(StubHub company website) AND (Organization OR Board OR Our Team OR Directors OR Leadership OR Management)', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop', 'nfpr': '1'}, 'search_information': {'query_displayed': '(StubHub company webs

In [25]:
import requests
from bs4 import BeautifulSoup
link = "https://www.bloomberg.com/news/articles/2024-03-01/new-world-development-sells-hong-kong-mall-for-514-million"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54'
}

def get_bloomberg_article(link: str):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54'
    }
    try:
        response = requests.get(link, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        title_tag = soup.find('title')
        title = title_tag.get_text(strip=True) if title_tag else "No title found"
        # Extract the content
        content = []
        for paragraph in soup.select('p[class^="media"]'):
            content.append(paragraph.get_text(strip=False))
        content_text = "\n".join(content) if content else "No content found"

        return title, content_text

    except Exception as e:
        print(f"Error fetching content from {link}: {e}")
        return None, None

# Example usage
title, content = fetch_bloomberg_article(link)
print("Title:", title)
print("Content:", content)

Title: New World Development Sells Hong Kong Mall D-PARK for $514 Million - Bloomberg
Content: New World Development Co. sold a shopping mall for HK$4.02 billion ($514 million) as part of its plan to improve financial health by offloading non-core assets.
The company sold all of its interest in D-PARK and its parking space in the Tsuen Wan area in Hong Kong to local developer Chinachem Group, it said in a statement on Friday. The company will continue disposal of assets, it added.


In [24]:
title_tag = soup.find('title')
title = title_tag.get_text(strip=True) if title_tag else "No title found"
# Extract the content
content = []
for paragraph in soup.select('p[class^="media"]'):
    content.append(paragraph.get_text(strip=False))
content_text = "\n".join(content) if content else "No content found"
print(content_text)

New World Development Co. sold a shopping mall for HK$4.02 billion ($514 million) as part of its plan to improve financial health by offloading non-core assets.
The company sold all of its interest in D-PARK and its parking space in the Tsuen Wan area in Hong Kong to local developer Chinachem Group, it said in a statement on Friday. The company will continue disposal of assets, it added.


In [16]:
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>New World Development Sells Hong Kong Mall D-PARK for $514 Million - Bloomberg</title><link href="https://www.bloomberg.com/favicon-black.png" rel="icon" type="image/png"/><link href="https://www.bloomberg.com/news/articles/2024-03-01/new-world-development-sells-hong-kong-mall-for-514-million" rel="canonical"/><meta content="width=device-width, initial-scale=1, viewport-fit=cover" name="viewport"/><meta content="app-id=281941097" name="apple-itunes-app"/><meta content="max-image-preview:large" name="robots"/><meta content="New World Development Co. sold a shopping mall for HK$4.02 billion ($514 million) as part of its plan to improve financial health by offloading non-core assets." name="description"/><meta content="New World Development Co. sold a shopping mall for HK$4.02 billion ($514 million) as part of its plan to improve financial health by offloading non-core a

In [2]:
for result in search_results:
    ner_results = nlp(result)
    full_name = []
    for item in ner_results:
        #print(item.keys())
        if item['entity_group'] == 'PER' and item['score'] >= 0.90:# and '#' not in item['word']:
            full_name.append(item['word'])
    print(full_name)
    print("\n ***************************")

NameError: name 'search_results' is not defined

In [57]:
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Initialize BERT NER pipeline
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def extract_management_info(text, nlp):
    # Define common position titles
    position_titles = [
        "CEO", "Chief Executive Officer", 
        "CFO", "Chief Financial Officer", 
        "COO", "Chief Operating Officer", 
        "CTO", "Chief Technology Officer", 
        "CMO", "Chief Marketing Officer", 
        "CBO", "Chief Business Officer", 
        "Director", "Vice President", "Board Member",
        "VP", "President", "Chairman", "Founder", 
        "Owner", "Partner", "Head", "Independent Director"
    ]
    
    # Create a regex pattern to find names and positions
    name_pattern = r'([A-Z][a-z]+(?: [A-Z][a-z]*){0,2})'
    position_pattern = r'\b(' + '|'.join(position_titles) + r')\b'
    combined_pattern = re.compile(rf'{name_pattern}\s*,?\s*{position_pattern}|{position_pattern}\s*,?\s*{name_pattern}', re.IGNORECASE)
    
    # Find all matches in the text
    matches = combined_pattern.findall(text)
    
    # Extract names and positions
    management_info = []
    for match in matches:
        filtered_match = [m for m in match if m]
        if len(filtered_match) == 2:
            name, position = (filtered_match[0].strip(), filtered_match[1].strip())
            # Validate the name using BERT NER
            ner_results = nlp(name)
            full_name = []
            for item in ner_results:
                #print(item.keys())
                print(item)
                if item['entity_group'] == 'PER' and item['score'] >= 0.90:# and '#' not in item['word']:
                    full_name.append(item['word'])
            if full_name:
                management_info.append((' '.join(full_name), position))
    return management_info

def clean_string(input_string):
    # Use regex to replace non-alphanumeric characters with an empty string
    cleaned_string = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return cleaned_string

def find_full_word(text, substring):
    # Find the first occurrence of the case-sensitive substring in the text using regex
    pattern = r'\b\w*' + re.escape(substring) + r'\w*\b'
    match = re.search(pattern, text)
    
    if match:
        # Return the full word containing the substring
        return match.group(0)
    else:
        return None

# Example usage
text = "This is an example sentence. Let's find the Substring in this text."
substring = "Sub"
result = find_full_word(text, substring)
print(result)  # Output: Substring

name_position_pairs = []
for key in search_results:
    text = clean_string(key)
    #print(text)
    management_info = extract_management_info(text, nlp)
    #print(management_info)
    name_position_pairs.extend(management_info)
#print(name_position_pairs)
# Count occurrences of each name-position pair
pair_counter = Counter(name_position_pairs)

# Find the most common pairs
most_common_pairs = pair_counter.most_common()

# print the most common name-position pairs
for pair, count in most_common_pairs:
    print(f"Name: {pair[0]}, Position: {pair[1]}, Count: {count}")



/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity_group': 'PER', 'score': 0.9991996, 'word': 'Art Yegorov', 'start': 9, 'end': 20}
{'entity_group': 'PER', 'score': 0.9996076, 'word': 'C', 'start': 0, 'end': 1}
{'entity_group': 'PER', 'score': 0.9953915, 'word': '##risto', 'start': 1, 'end': 6}
{'entity_group': 'PER', 'score': 0.7530372, 'word': '##pher Miller', 'start': 6, 'end': 17}
{'entity_group': 'PER', 'score': 0.9991628, 'word': 'Na', 'start': 4, 'end': 6}
{'entity_group': 'PER', 'score': 0.9269295, 'word': '##ku', 'start': 6, 'end': 8}
{'entity_group': 'PER', 'score': 0.9974962, 'word': '##l Paul', 'start': 8, 'end': 14}
{'entity_group': 'PER', 'score': 0.9994564, 'word': 'Julie Jacob', 'start': 5, 'end': 16}
{'entity_group': 'PER', 'score': 0.98797685, 'word': 'Na', 'start': 5, 'end': 7}
{'entity_group': 'PER', 'score': 0.94521886, 'word': '##l Paul', 'start': 9, 'end': 15}
{'entity_group': 'PER', 'score': 0.8243458, 'word': 'Yegorov', 'start': 9, 'end': 16}
{'entity_group': 'ORG', 'score': 0.7328082, 'word': 'Megan R

In [3]:
"""Pick server and start connection with VPNGate (http://www.vpngate.net/en/)"""

import requests, os, sys, tempfile, subprocess, base64, time

__author__ = "Andrea Lazzarotto"
__copyright__ = "Copyright 2014+, Andrea Lazzarotto"
__license__ = "GPLv3"
__version__ = "1.0"
__maintainer__ = "Andrea Lazzarotto"
__email__ = "andrea.lazzarotto@gmail.com"


if len(sys.argv) != 2:
    print('usage: ' + sys.argv[0] + ' [country name | country code]')
    exit(1)
country = 'Singapore'

if len(country) == 2:
    i = 6 # short name for country
elif len(country) > 2:
    i = 5 # long name for country
else:
    print('Country is too short!')
    exit(1)

try:
    vpn_data = requests.get('http://www.vpngate.net/api/iphone/').text.replace('\r','')
    servers = [line.split(',') for line in vpn_data.split('\n')]
    labels = servers[1]
    labels[0] = labels[0][1:]
    servers = [s for s in servers[2:] if len(s) > 1]
except:
    print('Cannot get VPN servers data')
    exit(1)

desired = [s for s in servers if country.lower() in s[i].lower()]
found = len(desired)
print('Found ' + str(found) + ' servers for country ' + country)
if found == 0:
    exit(1)

supported = [s for s in desired if len(s[-1]) > 0]
print(str(len(supported)) + ' of these servers support OpenVPN')
# We pick the best servers by score
winner = sorted(supported, key=lambda s: float(s[2].replace(',','.')), reverse=True)[0]

print("\n== Best server ==")
pairs = zip(labels, winner)[:-1]
for (l, d) in pairs[:4]:
    print(l + ': ' + d)

print(pairs[4][0] + ': ' + str(float(pairs[4][1]) / 10**6) + ' MBps')
print("Country: " + pairs[5][1])

print("\nLaunching VPN...")
_, path = tempfile.mkstemp()

f = open(path, 'w')
f.write(base64.b64decode(winner[-1]))
f.write('\nscript-security 2\nup /etc/openvpn/update-resolv-conf\ndown /etc/openvpn/update-resolv-conf')
f.close()

x = subprocess.Popen(['sudo', 'openvpn', '--config', path])

try:
    while True:
        time.sleep(600)
# termination with Ctrl+C
except:
    try:
        x.kill()
    except:
        pass
    while x.poll() != 0:
        time.sleep(1)
    print('\nVPN terminated')

Found 0 servers for country Singapore
0 of these servers support OpenVPN


IndexError: list index out of range